In [1]:
from src.extract import extract
from src.export import export, export_dir, display_progress

from slippi import Game

from pymongo import MongoClient
from bson.binary import Binary
from scipy import sparse
import numpy as np
import pickle
import os

In [15]:
database_name = 'slippi'
collection_name = 'CuckDaddy'
game_id = 'Game_20200918T203825.slp'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [16]:
# retreive player data on single game from database
p1, p2 = tuple(collection.find({'game_id':game_id}))[:2]

In [17]:
# convert istream from "sparse matrix bytestream" to numpy array
print(type(p1['istream']))
if isinstance(p1['istream'], bytes):
    p1['istream'] = pickle.loads( p1['istream']).toarray()
    p2['istream'] = pickle.loads( p2['istream']).toarray()
print(type(p1['istream']))

<class 'bytes'>
<class 'numpy.ndarray'>


In [18]:
print(f"{p1['name']} ({p1['code']}): {p1['character']}")
print(f"{p2['name']} ({p2['code']}): {p2['character']}")

Goloche (GOLO#831): CAPTAIN_FALCON
CuckDaddy (CUCK#434): FALCO


In [19]:
def presses(buttonstream):
    return int(np.sum(np.abs(np.diff(buttonstream, axis=0)))/2)

In [20]:
resultA = presses(p1['istream'][:, 9]) > presses(p2['istream'][:, 9])
resultB = presses(p1['istream'][:, 8]) > presses(p2['istream'][:, 8])
resultXY = presses(p1['istream'][:, [6,7]]) > presses(p2['istream'][:, [6,7]])
resultLR = presses(p1['istream'][:, [10,11]]) > presses(p2['istream'][:, [10,11]])
resultZ = presses(p1['istream'][:, 12]) > presses(p2['istream'][:, 12])

whoA = p1 if resultA else p2
whoB = p1 if resultB else p2
whoXY = p1 if resultXY else p2
whoLR = p1 if resultLR else p2
whoZ = p1 if resultZ else p2

print(f"{p1['name']}: {p1['character']}")
print('---VS---')
print(f"{p2['name']}: {p2['character']}")

print(f"\nMatch Length: {round(len(p1['istream'])/3600, 2)} minutes")
print(f"Match ID: {p1['game_id']}")

print(f"\nMost A presses: {presses(whoA['istream'][:, 9])}\n --- {whoA['name']} - {whoA['character']}")
print(f"\nMost B presses: {presses(whoB['istream'][:, 8])}\n --- {whoB['name']} - {whoB['character']}")
print(f"\nMost X or Y presses: {presses(whoXY['istream'][:, [6,7]])}\n --- {whoXY['name']} - {whoXY['character']}")
print(f"\nMost L or R presses: {presses(whoLR['istream'][:, [10,11]])}\n --- {whoLR['name']} - {whoLR['character']}")
print(f"\nMost Z presses: {presses(whoZ['istream'][:, 12])}\n --- {whoZ['name']} - {whoZ['character']}")

Goloche: CAPTAIN_FALCON
---VS---
CuckDaddy: FALCO

Match Length: 1.51 minutes
Match ID: Game_20200918T203825.slp

Most A presses: 54
 --- CuckDaddy - FALCO

Most B presses: 34
 --- CuckDaddy - FALCO

Most X or Y presses: 76
 --- CuckDaddy - FALCO

Most L or R presses: 57
 --- CuckDaddy - FALCO

Most Z presses: 2
 --- CuckDaddy - FALCO


In [21]:
if presses(p1['istream'][:, 6]) > presses(p1['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p1['name']} uses {jump_button} to jump")

if presses(p2['istream'][:, 6]) > presses(p2['istream'][:, 7]):
    jump_button = 'Y'
else:
    jump_button = 'X'
print(f"{p2['name']} uses {jump_button} to jump")

Goloche uses X to jump
CuckDaddy uses Y to jump


In [23]:
database_name = 'slippi'
collection_name = 'gh0st'

# Connect to the hosted MongoDB instance
client = MongoClient('localhost', 27017)
db = client[database_name]
collection = db[collection_name]

In [24]:
collection.count_documents({'code' : 'GOOS#845'})

0

In [25]:
collection.count_documents({'code' : 'GHST#265'})

283

In [26]:
collection.count_documents({'code' : 'CUCK#434'})

0